In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import GradientBoostingRegressor


In [3]:
df = pd.read_parquet("processed_data/final_with_anomalies.parquet")
store_features = pd.read_parquet("processed_data/store_features.parquet")
store_features.head()

,Store,mean_sales,sales_std,avg_pct_dev,if_anomaly_rate,high_conf_anomaly_rate,markdown_rate,holiday_sales,store_size,Cluster_KMeans3,Cluster_KMeans5,Cluster_pt_KMeans6,HC_Cluster5,Cluster_pt_KMeans2,HC_Cluster3
0,1,21710.543621,27748.945511,0.005136,0.051738,0.000976,0.356989,0.070578,151315,0,1,4,2,1,1
1,2,26898.070031,33077.612059,0.012600,0.061340,0.000195,0.357785,0.070522,202307,0,0,4,2,1,1
2,3,6373.033983,14251.034807,0.006354,0.018814,0.001439,0.359119,0.070053,37392,1,3,2,5,0,3
3,4,29161.210415,34583.677814,0.003083,0.080997,0.001266,0.357866,0.070775,205863,0,0,4,2,1,1
4,5,5053.415813,8068.221050,0.005480,0.006890,0.000889,0.358151,0.071119,34875,1,3,3,3,0,2


In [4]:
store_cluster_map = store_features.loc[:,["Store" , "Cluster_pt_KMeans6"]]
# rename the cluster column to Cluster
store_cluster_map = store_cluster_map.rename(columns={"Cluster_pt_KMeans6": "Cluster"})


In [5]:
df.shape

(421570, 29)

In [6]:
df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,Rolling_Mean_4,Rolling_Std_4,Z_Score,Pct_Deviation,Lag_1,Type_B,Type_C,IF_Anomaly,LOF_Anomaly,High_Confidence_Anomaly
0,1,1,2010-02-05,24924.50,0,42.31,2.572,0.0,0.0,0.0,...,24924.500000,0.000000,NaN,0.000000,24924.50,False,False,0,0,0
1,1,1,2010-02-12,46039.49,1,38.51,2.548,0.0,0.0,0.0,...,35481.995000,14930.552614,0.707107,0.297545,24924.50,False,False,1,0,0
2,1,1,2010-02-19,41595.55,0,39.93,2.514,0.0,0.0,0.0,...,37519.846667,11131.900957,0.366128,0.108628,46039.49,False,False,0,0,0
3,1,1,2010-02-26,19403.54,0,46.63,2.561,0.0,0.0,0.0,...,32990.770000,12832.106391,-1.058846,-0.411849,41595.55,False,False,0,0,0
4,1,1,2010-03-05,21827.90,0,46.50,2.625,0.0,0.0,0.0,...,32216.620000,13554.047185,-0.766466,-0.322465,19403.54,False,False,0,0,0


In [7]:
df = df.merge(store_cluster_map, on="Store", how="left")
df.shape

(421570, 30)

In [8]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 30 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   Store                    421570 non-null  int64         
 1   Dept                     421570 non-null  int64         
 2   Date                     421570 non-null  datetime64[ns]
 3   Weekly_Sales             421570 non-null  float64       
 4   IsHoliday                421570 non-null  int64         
 5   Temperature              421570 non-null  float64       
 6   Fuel_Price               421570 non-null  float64       
 7   MarkDown1                421570 non-null  float64       
 8   MarkDown2                421570 non-null  float64       
 9   MarkDown3                421570 non-null  float64       
 10  MarkDown4                421570 non-null  float64       
 11  MarkDown5                421570 non-null  float64       
 12  CPI                      42

In [ ]:
target = "Weekly_Sales"
feature_cols = []